In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score
import pandas as pd

## Data Loading and preperation

In [ ]:
def show_image(inputImage):
    newImage = inputImage.transpose(1, 2, 0) + 0.5
    plt.imshow(newImage[:,:,0])
    plt.show()

In [ ]:
def getData(filepath):
    df = pd.read_csv(filepath)
    x = df[df.columns[1:]].to_numpy()/255 - 0.5
    x = x.reshape(x.shape[0], 1, 28, 28)
    l = df['label'].to_numpy()
    return x,l

In [ ]:
X_train,y_train = getData('data/fashionmnist/fashion-mnist_train.csv')
X_test,y_test = getData('data/fashionmnist/fashion-mnist_test.csv')

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(X_train[0,:,:,:].shape)
show_image(X_train[7,:,:,:])

In [ ]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.len = len(data)
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        image = torch.tensor(self.data[index]).float()
        label = torch.tensor(self.labels[index]).long()
        return (image, label)

    def __len__(self):
        return self.len

In [ ]:
batchSize = 4
train_loader = torch.utils.data.DataLoader(dataset = customDataset(X_train, y_train),
                                           batch_size = batchSize, 
                                           shuffle = True)

## Creating Model

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.cnn_layers = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 1, out_channels = 4, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            torch.nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        
        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(7*7*8, 10)
        )

    def forward(self,x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1) # Flattening the input
        x = self.linear_layers(x)
        return x

In [ ]:
model = Net()
print(model)

## Defining Loss, Optimizer and Learning Rate

In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()

## Moving models to GPU

In [ ]:
if torch.cuda.is_available():
    model = model.cuda()
    loss_fn = loss_fn.cuda()

## Training the actual model

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
_,predicted = torch.max(model( torch.from_numpy(X_test).float().cuda() ),1)
predicted = predicted.cpu().numpy()
print('Test Accuracy : ', round(accuracy_score(y_true = y_test, y_pred = predicted) * 100,2))
torch.cuda.empty_cache()

## Saving Model

In [ ]:
PATH = 'data/fashionmnist/runs/fashion_mnist_net.pth'
torch.save(model.state_dict(), PATH)